<a href="https://colab.research.google.com/github/Breno-Crespo/EmbeddingGoogleIA/blob/main/Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [2]:
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get("Secret_KEY")
genai.configure(api_key=api_key)


In [3]:
for m in genai.list_models():
  if "embedContent" in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [4]:
#Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
      "\n"
      "Artigo completo: \n"
      "/n"
      "Gemini API & Google IA Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content= sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.032810617, -0.01688516, -0.025215462, 0.012716089, 0.08277083, 0.002536482, -0.026414808, -0.028317697, 0.05425331, 0.060539395, 0.0032525389, 0.020191211, -0.06561865, -0.03331605, 0.0022030266, -0.014546607, 0.02060882, -0.031048136, -0.02392801, -0.008800286, -0.006641589, 0.018742502, -0.030467432, -0.06449863, -0.029832648, 0.023105752, 0.011358189, -0.048242074, -0.02784867, 0.02181013, -0.054926004, 0.07345693, -0.037571188, 0.011125996, -0.052145366, -0.019738391, -0.030830065, -0.054747593, -0.005767497, 0.0059930095, 0.0022013078, -0.08643313, -0.009511407, 0.02499003, 0.0011687883, -0.034340013, 0.0424404, 0.030926676, 0.016177792, -0.06260421, 0.019944051, 0.018233264, 0.041917916, -0.038107622, -0.019975396, 0.001445452, 0.008838853, -0.050690122, 0.015546012, -0.012303607, 0.025936374, 0.027142111, -0.0056727002, 0.050826292, 0.021846468, -0.071234986, -0.028845657, -0.009066295, 0.0069035245, 0.04357643, 0.0021715092, -0.0029426343, 0.0488139, -0.0582867

In [5]:
# Listagem de documentos que serão buscados

DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema"
}

DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento"
}

DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada"
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [7]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [8]:
model = "models/embedding-001"

In [10]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [12]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"],row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.010924981, -0.016569521, -0.055676658, 0.0..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.015881274, -0.043948032, -0.015855772, -0.0..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[-0.00048746992, -0.03862515, 0.0015810853, 0...."


In [13]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]),embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [15]:
consulta = "Como faço para trocar marchas em um carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)

print(trecho)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada


In [24]:
generation_config = {
    "temperature": 0,
    "candidate_count": 1,
}

In [25]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Seu Googlecar é tipo um carro automático maneiro! Pra trocar as marchas, é só dar um toque na alavanca e escolher a que você quer. Fácil, né?
